In [1]:
# imports
import re
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize


from sklearn.model_selection import train_test_split

In [2]:
# use nltk to remove stopwords and lemmatize
# you might need to run: nltk.download() to fetch the stopword package in "all packages"
# you might also need to run ntlk.download("punkt")
english_stopwords = set(stopwords.words("english"))
wordnet_lemmatizer = WordNetLemmatizer()

In [62]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [3]:
def clean_input_text(text):
    clean_text = []
    for sent in text:
        clean_sent = ""
        sent_tokens = word_tokenize(sent)
        for token in sent_tokens:
            clean_sent += wordnet_lemmatizer.lemmatize(token) + " " if token not in english_stopwords else ""
        clean_text.append(clean_sent)
    return clean_text

In [5]:
#  convert the labels to vectors of binary data
dic_labels = {
    "NUM": 0,
    "LOC": 1,
    "HUM": 2,
    "DESC": 3,
    "ENTY": 4,
    "ABBR": 5,
    "NUM": 6
}

In [12]:
def load_data_questions_dataset(filename):
    examples = list(open(filename, "r", encoding="utf-8").readlines())
    examples = [s.strip() for s in examples]
    # the label is the first part of the string before the semicolon
    xs, labels = [], []
    for example in examples:
        split = example.split(":")
        labels.append(dic_labels[split[0]])
        xs.append(split[1])
    return xs, labels

test_filename = "./data/questions/questions_test.txt"
train_filename = "./data/questions/questions_train.txt"
x1, y1 = load_data_questions_dataset(test_filename)
size_test = len(x1)
x2, y2 = load_data_questions_dataset(train_filename)
x_text = clean_input_text(x1 + x2)
y = y1 + y2

In [8]:
# hyperparameters for the cnn dealing with the movie dataset
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2
NUMBER_DIFFERENT_OUTPUTS = 7

In [9]:
# load pretrain glove word2vec instance for preprocessing
filename = './data/glove.6B.300d.txt'

print('Indexing word vectors.')

embeddings_index = {}
with open(filename, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [10]:
# vectorize the input text (both negative and positive )
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(x_text)
sequences = tokenizer.texts_to_sequences(x_text)
word_index = tokenizer.word_index
print(len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
labels[0]

8076
Shape of data tensor: (5952, 1000)
Shape of label tensor: (5952, 7)


array([0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [13]:
x_val = data[:size_test]
y_val = labels[:size_test]

x_train = data[size_test:]
y_train = labels[size_test:] 

In [14]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [15]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [17]:
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(NUMBER_DIFFERENT_OUTPUTS, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 5452 samples, validate on 500 samples
Epoch 1/10
5452/5452 [==============================] - 5s 916us/step - loss: 0.5499 - acc: 0.8063 - val_loss: 0.1486 - val_acc: 0.9540
Epoch 2/10
5452/5452 [==============================] - 3s 478us/step - loss: 0.0944 - acc: 0.9707 - val_loss: 0.1159 - val_acc: 0.9500
Epoch 3/10
5452/5452 [==============================] - 3s 477us/step - loss: 0.0378 - acc: 0.9884 - val_loss: 0.1433 - val_acc: 0.9540
Epoch 4/10
5452/5452 [==============================] - 3s 480us/step - loss: 0.0297 - acc: 0.9916 - val_loss: 0.0767 - val_acc: 0.9760
Epoch 5/10
5452/5452 [==============================] - 3s 479us/step - loss: 0.0141 - acc: 0.9947 - val_loss: 0.0963 - val_acc: 0.9740
Epoch 6/10
5452/5452 [==============================] - 3s 480us/step - loss: 0.0091 - acc: 0.9976 - val_loss: 0.0777 - val_acc: 0.9760
Epoch 7/10
5452/5452 [==============================] - 3s 477us/step - loss: 0.0101 - acc: 0.9972 - val_loss: 0.0713 - val_acc: 0.9780
E